# OpenVINO Model Server in OpenShift demo

This notebook demonstrate how to deploy and use OpenVINO Model Server.
That will include the use case with BERT model and a pipeline performing face detection operation and also age, gender and emotion recognition for each detected face.

Requirements:
- OpenShift cluster with the API access to a project. In the demo `ovms` project is used.
- installed [OpenVINO Model Server Operator](https://catalog.redhat.com/software/operators/search?q=openvino)
- Jupyter session with python3 deployed in the cluster

## Creating Minio storage

OpenVINO Model Server can expose over gRPC and REST interface the models stored in the local or cloud storage like AWS S3, google storage or Azure blobs. In OpenShift and Kubernetes every Persistent Storage Claim could be used as well. In this demo will be employed Minio service which is an equivalent of AWS S3.

First login to OpenShift cluster API using `oc` tool. In the commands below change the cluster DNS name and the user token.

In [1]:
!oc login --token=<user token> --server=https://api.<cluster dns name>:6443

Logged into "https://api.openvino5.3q12.p1.openshiftapps.com:6443" as "dtrawins" using the token provided.

You have access to 100 projects, the list has been suppressed. You can list all projects with 'oc projects'

Using project "ovms".


Change the project context where you would like to deploy your services.

In [2]:
!oc project ovms

Already on project "ovms" on server "https://api.openvino5.3q12.p1.openshiftapps.com:6443".


Now deploy Minio service. Note that the configuration below creates Minio server with emphemeral storage which will be deleted each time the pod is restarted. It includes also the default credentials. All in all, it is only a demonstrative purpose.

In [3]:
!oc apply -f https://raw.githubusercontent.com/openvinotoolkit/openvino_notebooks/ovms-demo/notebooks/202-model-server/minio.yaml

deployment.apps/minio created
service/minio-service created


Next step is to connect to the Minio service and create models repository for the OpenVINO Model Server

In [4]:
!wget https://dl.min.io/client/mc/release/linux-amd64/mc

--2021-04-22 20:49:32--  https://dl.min.io/client/mc/release/linux-amd64/mc
Resolving dl.min.io (dl.min.io)... 178.128.69.202
Connecting to dl.min.io (dl.min.io)|178.128.69.202|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 20529152 (20M) [application/octet-stream]
Saving to: ‘mc’

mc                  100%[===================>]  19.58M  15.0MB/s    in 1.3s    

2021-04-22 20:49:34 (15.0 MB/s) - ‘mc’ saved [20529152/20529152]



In [5]:
!chmod 755 mc

In the command below make sure you have the correct project name. Replace `ovms` with your project name, where minio got deployed.

In [6]:
!./mc alias set minio http://minio-service.ovms:9000 minio minio123

Added `minio` successfully.


In [7]:
!./mc mb minio/models

Bucket created successfully `minio/models`.


## Creating models repository

While the Minio is available, we can upload the models for serving in the OpenVINO Model Server. In the demos below will be needed 4 models:
- [resnet](https://github.com/onnx/models/tree/master/vision/classification/resnet)
- [face detection](https://github.com/openvinotoolkit/open_model_zoo/blob/master/models/intel/face-detection-retail-0004/description/face-detection-retail-0004.md)
- [age-gender recognition](https://github.com/openvinotoolkit/open_model_zoo/blob/master/models/intel/age-gender-recognition-retail-0013/description/age-gender-recognition-retail-0013.md)
- [emotion recognition](https://github.com/openvinotoolkit/open_model_zoo/blob/master/models/intel/emotions-recognition-retail-0003/description/emotions-recognition-retail-0003.md)

In [8]:
!curl --create-dirs https://storage.openvinotoolkit.org/repositories/open_model_zoo/2021.3/models_bin/2/age-gender-recognition-retail-0013/FP32/age-gender-recognition-retail-0013.xml -o age-gender/1/age-gender-recognition-retail-0013.xml 
!curl --create-dirs https://storage.openvinotoolkit.org/repositories/open_model_zoo/2021.3/models_bin/2/age-gender-recognition-retail-0013/FP32/age-gender-recognition-retail-0013.bin -o age-gender/1/age-gender-recognition-retail-0013.bin
!curl --create-dirs https://storage.openvinotoolkit.org/repositories/open_model_zoo/2021.3/models_bin/2/face-detection-retail-0004/FP32/face-detection-retail-0004.xml -o face-detection/1/face-detection-retail-0004.xml
!curl --create-dirs https://storage.openvinotoolkit.org/repositories/open_model_zoo/2021.3/models_bin/2/face-detection-retail-0004/FP32/face-detection-retail-0004.bin -o face-detection/1/face-detection-retail-0004.bin
!curl --create-dirs https://storage.openvinotoolkit.org/repositories/open_model_zoo/2021.3/models_bin/2/emotions-recognition-retail-0003/FP32/emotions-recognition-retail-0003.xml -o emotions/1/emotions-recognition-retail-0003.xml
!curl --create-dirs https://storage.openvinotoolkit.org/repositories/open_model_zoo/2021.3/models_bin/2/emotions-recognition-retail-0003/FP32/emotions-recognition-retail-0003.bin -o emotions/1/emotions-recognition-retail-0003.bin
!curl -L --create-dir https://github.com/onnx/models/raw/master/vision/classification/resnet/model/resnet50-caffe2-v1-9.onnx -o resnet/1/resnet50-caffe2-v1-9.onnx

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 30901  100 30901    0     0  48132      0 --:--:-- --:--:-- --:--:-- 48132
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 8351k  100 8351k    0     0  8318k      0  0:00:01  0:00:01 --:--:-- 8318k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  101k  100  101k    0     0   969k      0 --:--:-- --:--:-- --:--:--  969k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 2297k  100 2297k    0     0  2927k      0 --:--:-- --:--:-- --:--:-- 2927k
  % Total    % Received % Xferd  Average Speed   Tim

In [9]:
!./mc cp --recursive age-gender minio/models/
!./mc cp --recursive face-detection minio/models/
!./mc cp --recursive emotions minio/models/
!./mc cp --recursive resnet minio/models/

...v1-9.onnx:  97.74 MiB / 97.74 MiB ┃▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓┃ 580.17 MiB/s 0s

In [10]:
!./mc ls -r minio/models/

[2021-04-22 20:50:28 UTC] 8.2MiB age-gender/1/age-gender-recognition-retail-0013.bin
[2021-04-22 20:50:28 UTC]  30KiB age-gender/1/age-gender-recognition-retail-0013.xml
[2021-04-22 20:50:29 UTC] 9.5MiB emotions/1/emotions-recognition-retail-0003.bin
[2021-04-22 20:50:29 UTC]  38KiB emotions/1/emotions-recognition-retail-0003.xml
[2021-04-22 20:50:28 UTC] 2.2MiB face-detection/1/face-detection-retail-0004.bin
[2021-04-22 20:50:28 UTC] 102KiB face-detection/1/face-detection-retail-0004.xml
[2021-04-22 20:50:30 UTC]  98MiB resnet/1/resnet50-caffe2-v1-9.onnx


With the model repository created, we can move on the deploying OpenVINO Model Server in the cluster.

## OpenVINO Model Server deployment with a single model

The first scenario will be with a serving a single model. In the demo, there will be performed image classification using ResNet50 model in ONNX format.

While the operator in place, starting the inference service is easy:

In [11]:
!curl -s https://raw.githubusercontent.com/dtrawins/openvino_notebooks/ovms-demo/notebooks/202-model-server/ovms-resnet.yaml

apiVersion: intel.com/v1alpha1
kind: Ovms
metadata:
  name: ovms-resnet
spec:
  aws_access_key_id: "minio"
  aws_region: "us-east-1"
  aws_secret_access_key: "minio123"
  grpc_port: 8080
  image_name: registry.connect.redhat.com/intel/openvino-model-server:latest
  log_level: INFO
  model_name: "resnet"
  model_path: "s3://minio-service:9000/models/resnet"
  plugin_config: '{\"CPU_THROUGHPUT_STREAMS\":\"1\"}'
  replicas: 1
  resources:
    limits:
      cpu: 4
      memory: 500Mi
  rest_port: 8081
  service_type: ClusterIP


In [12]:
!oc apply -f https://raw.githubusercontent.com/dtrawins/openvino_notebooks/ovms-demo/notebooks/202-model-server/ovms-resnet.yaml

ovms.intel.com/ovms-resnet created


In [13]:
!oc get pod
!oc get service

NAME                           READY     STATUS    RESTARTS   AGE
minio-5c57f888dd-69dzq         1/1       Running   0          4m42s
ovms-resnet-7cdb696f7b-pxl7z   1/1       Running   0          32s
NAME            TYPE        CLUSTER-IP       EXTERNAL-IP   PORT(S)             AGE
minio-service   ClusterIP   172.30.157.153   <none>        9000/TCP            4m43s
ovms-resnet     ClusterIP   172.30.88.10     <none>        8080/TCP,8081/TCP   34s


With those steps, OpenVINO Model Server is running and is ready to accept inference requests. The status of models can be queries with a simple REST API calls:

In [14]:
!curl http://ovms-resnet.ovms.svc:8081/v1/models/resnet

{
 "model_version_status": [
  {
   "version": "1",
   "state": "AVAILABLE",
   "status": {
    "error_code": "OK",
    "error_message": "OK"
   }
  }
 ]
}


In [15]:
!curl http://ovms-resnet.ovms.svc:8081/v1/models/resnet/metadata

{
 "modelSpec": {
  "name": "resnet",
  "signatureName": "",
  "version": "1"
 },
 "metadata": {
  "signature_def": {
   "@type": "type.googleapis.com/tensorflow.serving.SignatureDefMap",
   "signatureDef": {
    "serving_default": {
     "inputs": {
      "gpu_0/data_0": {
       "dtype": "DT_FLOAT",
       "tensorShape": {
        "dim": [
         {
          "size": "1",
          "name": ""
         },
         {
          "size": "3",
          "name": ""
         },
         {
          "size": "224",
          "name": ""
         },
         {
          "size": "224",
          "name": ""
         }
        ],
        "unknownRank": false
       },
       "name": "gpu_0/data_0"
      }
     },
     "outputs": {
      "gpu_0/softmax_1": {
       "dtype": "DT_FLOAT",
       "tensorShape": {
        "dim": [
         {
          "size": "1",
          "name": ""
         },
         {
          "size": "1000",
          "name": ""
         }
        ],
        "unknownRank": false

## Running predition requests

Lets copy an axilary file with ImageNet class names:

In [16]:
!wget -q https://raw.githubusercontent.com/openvinotoolkit/model_server/main/example_client/classes.py

In [17]:
import grpc
import numpy as np
import classes
from tensorflow import make_tensor_proto, make_ndarray, make_tensor_proto
from tensorflow_serving.apis import predict_pb2
from tensorflow_serving.apis import prediction_service_pb2_grpc
import cv2



Next we create functions formating the array out of input image. The array will be transofrmed to required format and data range.


In [18]:
!wget -q https://github.com/openvinotoolkit/model_server/raw/main/example_client/images/bee.jpeg

In [19]:
def preprocess(img_data):
    mean_vec = np.array([0.485, 0.456, 0.406])
    stddev_vec = np.array([0.229, 0.224, 0.225])
    norm_img_data = np.zeros(img_data.shape).astype('float32')
    for i in range(img_data.shape[0]):
         # for each pixel in each channel, divide the value by 255 to get value between [0, 1] and then normalize
        norm_img_data[i,:,:] = (img_data[i,:,:]/255 - mean_vec[i]) / stddev_vec[i]
    return norm_img_data

def getJpeg(path, size):
    with open(path, mode='rb') as file:
        content = file.read()

    img = np.frombuffer(content, dtype=np.uint8)
    img = cv2.imdecode(img, cv2.IMREAD_COLOR)  # BGR format
    # format of data is HWC
    # add image preprocessing if needed by the model
    img = cv2.resize(img, (224, 224))
    img = img.astype('float32')
    #convert to RGB instead of BGR if required by model
    #img = img[:, :, [2, 1, 0]]
    #convert to NHWC
    img = img.transpose(2,0,1)
    # normalize to adjust to model training dataset
    img = preprocess(img)
    img = img.reshape(1,3,size,size)
    print(path, img.shape, "; data range:",np.amin(img),":",np.amax(img))
    return img



Let's try to classify this image:

![image](https://github.com/openvinotoolkit/model_server/raw/main/example_client/images/bee.jpeg)

In [20]:
img1 = getJpeg('bee.jpeg', 224)

channel = grpc.insecure_channel("ovms-resnet.ovms.svc:8080")
stub = prediction_service_pb2_grpc.PredictionServiceStub(channel)

request = predict_pb2.PredictRequest()
request.model_spec.name = "resnet"
request.inputs["gpu_0/data_0"].CopyFrom(make_tensor_proto(img1, shape=(img1.shape)))
result = stub.Predict(request, 10.0) # result includes a dictionary with all model outputs

output = make_ndarray(result.outputs["gpu_0/softmax_1"])
ma = np.argmax(output)
print("Class is with highest score: {}".format(ma))
print("Detected class name: {}".format(classes.imagenet_classes[ma]))

bee.jpeg (1, 3, 224, 224) ; data range: -2.117904 : 2.64
Class is with highest score: 309
Detected class name: bee


That concludes the demonstration of a simple inference request. In the following example, more advance use case will be presented.

## Deploying OpenVINO Model Server with a pipeline of models and a custom node implementation

In that part of the demo, we will configure, deploy and use the OpenVINO Model Server with a pipeline of models and a custom node.
![graph](https://raw.githubusercontent.com/openvinotoolkit/model_server/develop/docs/faces_analysis_graph.svg)

![graph](https://github.com/openvinotoolkit/model_server/raw/develop/docs/faces_analysis.png)

The models were uploaded to Minio earlier, so now the missing component is the custom node. Face detection model returns a set of 200 detection results. It includes the coordinate and the detection scores. To employ face analysis models we neeed to crop the detected faces, filter the results with low scores and resize the retreived face images to a correct target size.

The the functionality or custom node in OVMS Directed Acyclic Graph scheduler, everyone can create arbitrary implementation of the data transformation node in the pipeline. It will be attached to the model server as a dynamic library. Lets use such [example of the custom node](https://github.com/openvinotoolkit/model_server/tree/develop/src/custom_nodes/model_zoo_intel_object_detection).

That custom node library could be build with a simple `make` command and docker image. Here we will repeat the commands from the makefile and the [dockerfile](https://github.com/openvinotoolkit/model_server/blob/develop/src/custom_nodes/model_zoo_intel_object_detection/Dockerfile)

In [21]:
!git clone --depth=1 -b develop https://github.com/openvinotoolkit/model_server
!curl -s https://download.01.org/opencv/master/openvinotoolkit/thirdparty/linux/opencv/opencv_4.5.1-044_centos7.txz | tar --use-compress-program=xz -xf -

Cloning into 'model_server'...
remote: Enumerating objects: 585, done.
remote: Counting objects: 100% (585/585), done.
remote: Compressing objects: 100% (503/503), done.
remote: Total 585 (delta 155), reused 275 (delta 62), pack-reused 0
Receiving objects: 100% (585/585), 4.42 MiB | 39.72 MiB/s, done.
Resolving deltas: 100% (155/155), done.
tar: .: Cannot utime: Operation not permitted
tar: .: Cannot change mode to rwxr-sr-x: Operation not permitted
tar: Exiting with failure status due to previous errors


In [22]:
!g++ -c -std=c++17 model_server/src/custom_nodes/model_zoo_intel_object_detection/model_zoo_intel_object_detection.cpp -fpic  -I./opencv/include/ -Wall -Wno-unknown-pragmas -Werror -fno-strict-overflow -fno-delete-null-pointer-checks -fwrapv -fstack-protector
!g++ -shared -o libcustom_node.so model_zoo_intel_object_detection.o -L./opencv/lib/ -I./opencv/include/ -lopencv_core -lopencv_imgproc -lopencv_imgcodecs

In [23]:
!ls -l libcustom_node.so

-rwxr-xr-x. 1 1000890000 1000890000 212104 Apr 22 20:55 libcustom_node.so


With the commands above, we created the library `libcustom_node.so`.

We will need also the configuration file which defines the pipeline. 

In [24]:
!wget -q https://raw.githubusercontent.com/dtrawins/openvino_notebooks/ovms-demo/notebooks/202-model-server/config.json
!cat config.json    

{
  "model_config_list": [
    {"config": {
      "name": "face_detection",
      "base_path": "s3://models/face-detection/",
      "shape": "(1,3,400,600)"}},
    {"config": {
      "name": "age_gender_recognition",
      "base_path": "s3://models/age-gender/",
      "shape": "(1,3,64,64)"}},
    {"config": {
      "name": "emotion_recognition",
      "base_path": "s3://models/emotions/",
      "shape": "(1,3,64,64)"}}
  ],
  "custom_node_library_config_list": [
    {"name": "object_detection_image_extractor",
      "base_path": "/config/libcustom_node.so"}
  ],
  "pipeline_config_list": [
    {
      "name": "find_face_images",
      "inputs": [
        "image"
      ],
      "nodes": [
        {
          "name": "face_detection_node",
          "model_name": "face_detection",
          "type": "DL model",
          "inputs": [
            {"data": {
              "node_name": "request",
              "data_item": "image"}}],
          "outputs": [
            {"data_item": "detecti

Let's add the custom node library and the config.json to a ConfigMap resource. It will be mounted later inside the OpenVINO Model Server containers.

In [26]:
!oc create configmap ovms-face-detection-pipeline --from-file=libcustom_node.so=libcustom_node.so --from-file=config.json=config.json

configmap/ovms-face-detection-pipeline created


We are ready to deploy the model server using the following Ovms resource:

In [27]:
!curl -s https://raw.githubusercontent.com/dtrawins/openvino_notebooks/ovms-demo/notebooks/202-model-server/ovms-face-detection-pipeline.yaml

apiVersion: intel.com/v1alpha1
kind: Ovms
metadata:
  name: ovms-pipeline
  namespace: ovms
spec:
  aws_access_key_id: "minio"
  aws_region: "us-east-1"
  aws_secret_access_key: "minio123"
  s3_compat_api_endpoint: 'http://minio-service:9000'
  config_configmap_name: 'ovms-face-detection-pipeline'
  grpc_port: 8080
  image_name: >-
    registry.connect.redhat.com/intel/openvino-model-server:latest
  log_level: INFO
  plugin_config: '{\"CPU_THROUGHPUT_STREAMS\":\"1\"}'
  replicas: 1
  rest_port: 8081
  service_type: ClusterIP


In [28]:
!oc apply -f https://raw.githubusercontent.com/dtrawins/openvino_notebooks/ovms-demo/notebooks/202-model-server/ovms-face-detection-pipeline.yaml

ovms.intel.com/ovms-pipeline created


In [29]:
!oc get service
!oc get pod

NAME            TYPE        CLUSTER-IP       EXTERNAL-IP   PORT(S)             AGE
minio-service   ClusterIP   172.30.157.153   <none>        9000/TCP            11m
ovms-pipeline   ClusterIP   172.30.57.114    <none>        8080/TCP,8081/TCP   59s
ovms-resnet     ClusterIP   172.30.88.10     <none>        8080/TCP,8081/TCP   7m2s
NAME                             READY     STATUS    RESTARTS   AGE
minio-5c57f888dd-69dzq           1/1       Running   0          11m
ovms-pipeline-66494fbd96-2mhvz   1/1       Running   0          60s
ovms-resnet-7cdb696f7b-pxl7z     1/1       Running   0          7m2s


The model server with face detection pipeline is deployed we can test the status of models:

In [30]:
!curl -s http://ovms-pipeline.ovms.svc:8081/v1/config

E0422 21:00:18.315934811    4824 backup_poller.cc:133]       Run client channel backup poller: {"created":"@1619125218.315887516","description":"pollset_work","file":"src/core/lib/iomgr/ev_epollex_linux.cc","file_line":321,"referenced_errors":[{"created":"@1619125218.315843714","description":"Bad file descriptor","errno":9,"file":"src/core/lib/iomgr/ev_epollex_linux.cc","file_line":948,"os_error":"Bad file descriptor","syscall":"epoll_wait"}]}
{
"age_gender_recognition" : 
{
 "model_version_status": [
  {
   "version": "1",
   "state": "AVAILABLE",
   "status": {
    "error_code": "OK",
    "error_message": "OK"
   }
  }
 ]
},
"emotion_recognition" : 
{
 "model_version_status": [
  {
   "version": "1",
   "state": "AVAILABLE",
   "status": {
    "error_code": "OK",
    "error_message": "OK"
   }
  }
 ]
},
"face_detection" : 
{
 "model_version_status": [
  {
   "version": "1",
   "state": "AVAILABLE",
   "status": {
    "error_code": "OK",
    "error_message": "OK"
   }
  }
 ]
},
"find_

The pipeline execution is represented as the model `find_face_images`. The client run the prediction request exectly the same way like with the single models. Below the gRPC client will perform remote inference.

In [31]:
import grpc
import cv2
import os
import numpy as np
from tensorflow import make_tensor_proto, make_ndarray
import argparse
from tensorflow_serving.apis import predict_pb2
from tensorflow_serving.apis import prediction_service_pb2_grpc

Let's prepare the image to analyse
![image](https://github.com/openvinotoolkit/model_server/raw/develop/example_client/images/people/people2.jpeg)


In [32]:
!wget -q https://github.com/openvinotoolkit/model_server/raw/develop/example_client/images/people/people2.jpeg

In [33]:
img = cv2.imread('people2.jpeg').astype(np.float32)  # BGR color format, shape HWC
resolution = (400, 600)
img = cv2.resize(img, (resolution[1], resolution[0]))
img = img.transpose(2,0,1).reshape(1,3,resolution[0],resolution[1])
print(img.shape)

    
    

(1, 3, 400, 600)


Next we establish connection with the model server

In [187]:
address = "ovms-pipeline.ovms.svc:8080"
MAX_MESSAGE_LENGTH = 1024 * 1024 * 8  # incresed default max size of the message
channel = grpc.insecure_channel(address,
    options=[
        ('grpc.max_send_message_length', MAX_MESSAGE_LENGTH),
        ('grpc.max_receive_message_length', MAX_MESSAGE_LENGTH),
    ])

stub = prediction_service_pb2_grpc.PredictionServiceStub(channel)
request = predict_pb2.PredictRequest()
request.model_spec.name = "find_face_images"

Below is created the request and prediction gets executed. Note, that the exeption is handled when the pipeline don't detect any face in the image. 

In [191]:
request.inputs['image'].CopyFrom(make_tensor_proto(img, shape=img.shape))
try:
    response = stub.Predict(request, 10.0)
except grpc.RpcError as err:
    if err.code() == grpc.StatusCode.ABORTED:
        print('No face has been found in the image')
        exit(1)
    else:
        raise err


Results are in the `response` object in a set of outputs: `ages`, `genders`, `emotions`, `face_coordinates`. Ouput `face_images` returns cropped faces retreived from the original image.

In [227]:
def save_face_images_as_jpgs(output_nd, name, location):
    for i in range(output_nd.shape[0]):
        out = output_nd[i][0]
        out = out.transpose(1,2,0)
        cv2.imwrite(os.path.join(location, name + '_' + str(i) + '.jpg'), out)
        
def update_people_ages(output_nd, people):
    for i in range(output_nd.shape[0]):
        age = int(output_nd[i,0,0,0,0] * 100)
        if len(people) < i + 1:
            people.append({'age': age})
        else:
            people[i].update({'age': age})
    return people

def update_people_genders(output_nd, people):
    for i in range(output_nd.shape[0]):
        gender = 'male' if output_nd[i,0,0,0,0] < output_nd[i,0,1,0,0] else 'female'
        if len(people) < i + 1:
            people.append({'gender': gender})
        else:
            people[i].update({'gender': gender})
    return people

def update_people_emotions(output_nd, people):
    emotion_names = {
        0: 'neutral',
        1: 'happy',
        2: 'sad',
        3: 'surprised',
        4: 'angry'
    }
    for i in range(output_nd.shape[0]):
        emotion_id = np.argmax(output_nd[i,0,:,0,0])
        emotion = emotion_names[emotion_id]
        if len(people) < i + 1:
            people.append({'emotion': emotion})
        else:
            people[i].update({'emotion': emotion})
    return people

def update_people_coordinate(output_nd, people):
    for i in range(output_nd.shape[0]):
        if len(people) < i + 1:
            people.append({'coordinate': output_nd[i,0,:]})
        else:
            people[i].update({'coordinate': output_nd[i,0,:]})
    return people

In [228]:
people = []

for name in response.outputs:
    print(f"Output: name[{name}]")
    tensor_proto = response.outputs[name]
    output_nd = make_ndarray(tensor_proto)
    print(f"    numpy => shape[{output_nd.shape}] data[{output_nd.dtype}]")

    if name == 'face_images':
        save_face_images_as_jpgs(output_nd, name, ".")
    if name == 'ages':
        people = update_people_ages(output_nd, people)
    if name == 'genders':
        people = update_people_genders(output_nd, people)
    if name == 'emotions':
        people = update_people_emotions(output_nd, people)
    if name == 'face_coordinates':
        people = update_people_coordinate(output_nd, people)

Output: name[face_coordinates]
    numpy => shape[(2, 1, 4)] data[float32]
Output: name[genders]
    numpy => shape[(2, 1, 2, 1, 1)] data[float32]
Output: name[face_images]
    numpy => shape[(2, 1, 3, 64, 64)] data[float32]
Output: name[confidence_levels]
    numpy => shape[(2, 1, 1)] data[float32]
Output: name[ages]
    numpy => shape[(2, 1, 1, 1, 1)] data[float32]
Output: name[emotions]
    numpy => shape[(2, 1, 5, 1, 1)] data[float32]


In [223]:
print('\nFound', len(people), 'faces:')
for person in people:
    print('Age:', person['age'], '; Gender:', person['gender'], '; Emotion:', person['emotion'], '; Original image coordinate:', person['coordinate'])


Found 2 faces:
Age: 48 ; Gender: male ; Emotion: happy ; Original image coordinate: [0.4584167  0.11273344 0.6308887  0.4844094 ]
Age: 20 ; Gender: female ; Emotion: happy ; Original image coordinate: [0.11928874 0.5223113  0.25304735 0.75718385]


![person1](face_images_0.jpg)
![person2](face_images_1.jpg)

## Cleanup

In [229]:
!rm -Rf emotions
!rm -Rf age-gender
!rm -Rf face-detection
!rm -Rf resnet
!rm -Rf model_server
!rm -Rf opencv
!rm bee.jpeg classes.py config.json mc people* face_images* model_zoo_intel_object_detection* libcustom_node.so


In [ ]:
!oc delete Ovms ovms-resnet
!oc delete Ovms ovms-pipeline
!oc delete deploy minio
!oc delete service minio-service
!oc delete configmap ovms-face-detection-pipeline
